# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [16]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF


In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## Question 1

Since the features are categorical we can use the chi-squared test of independence for comparing proportions




In [17]:
data_tab = pd.crosstab(data.race,data.call, margins=True)
data_tab.columns = ["no_call", "call","row_totals"]
data_tab.index = ["black","white","col_totals"]


In [18]:
observed = data_tab.iloc[0:2,0:2]
data_tab

,no_call,call,row_totals
black,2278,157,2435
white,2200,235,2435
col_totals,4478,392,4870


In [27]:
# why
observed.sum().sum()

4870

In [21]:
expected =  np.outer(data_tab["row_totals"][0:2],
                     data_tab.loc["col_totals"][0:2]) / 4870

expected = pd.DataFrame(expected)

expected.columns = ["No Call", "Call"]
expected.index = ["Black","White"]

expected

,No Call,Call
Black,2239,196
White,2239,196


In [22]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

nan


In [23]:
from scipy.stats import chi2_contingency

In [24]:
chi2, p, dof, expected = chi2_contingency(observed)

In [26]:
print(chi2)
print(p)
print(dof)
print(expected)

16.4490285842
4.99757838996e-05
1
[[ 2239.   196.]
 [ 2239.   196.]]


### Question 2: What is the Null and Alternative Hypothesis?

Null: The null hypothesis is the commonly accepted hypothesis. In this situation the null hypothesis would be that the sound of a name has no affect on whether or not they will receive a call. So calls will be distributed 50% to white and 50% to black

Alternative: The alternative hypothesis is the idea of trying to prove the null hypothesis wrong. In this case the alternative hypothesis is that the sound of a name does affect whether or not they will receive a call. I'm going to propose that 60% of white received a call and 40% of black received a call.

### Question 3: Margin of error, confidence interval, and p-value